In [ ]:
f= read("federalistpapers.txt", String)

subEssay = String[]

i = 1
while i != nothing 
    next_occurrence_start = findnext("FEDERALIST No.", f, i)
    next_occurrence_end = findnext("PUBLIUS", f, i)
    println(SubString(f, next_occurrence_start[1], next_occurrence_end[2]))
    if next_occurrence_start == nothing
        break
    end
    i = next_occurrence_end[1] + 1
end


In [63]:
# TU PHAM
# Questions 1, 2, 4
text = read("federalistpapers.txt", String)
essays = split(text, r"FEDERALIST No.\ \d+")
popfirst!(essays)
authors = Array{String}(undef, length(essays))
for i in eachindex(essays)
    essayParts = split(essays[i], "PUBLIUS")
    essayParts = split(essayParts[1], "To the People of the State of New York")
    firstPart = split(essayParts[1], "\r\n\r\n")
    if(i in 49:58 || i in 62:63)
        authors[i] = "Unknown"
    else
        authors[i] = replace(firstPart[end - 1], "\r\n" => "")
    end
    secondPart = essayParts[2]
    essays[i] = string(secondPart[6:end], "PUBLIUS")
end
essays

# Question 3
function separateIntoParagraphs(essays)
    array = Array{String}[]
    for i in eachindex(essays)
        paragraphs = split(essays[i], "\r\n\r\n")
        j = 1
        while (j <= length(paragraphs))
            if(cmp(paragraphs[j], "") == 0)
                deleteat!(paragraphs, j)
            else
                paragraphs[j] = replace(paragraphs[j], "\r\n" => " ")
                j += 1
            end
        end
        push!(array, paragraphs)
    end
    return array
end

array = separateIntoParagraphs(essays)

# Questions 5, 6, 7, 8(incomplete)
function editText(text)
    editedText = lowercase(text)
    editedText = replace(editedText, r"\(\d+\)" => "")
    editedText = replace(editedText, r"\d+" => "")
    return editedText
end

function editParagraphs(array)
    for i in eachindex(array)
        for j in eachindex(array[i])
            array[i][j] = editText(array[i][j])
        end
    end
    return array
end

function editEssays(essays)
    for i in eachindex(essays)
        essays[i] = editText(essays[i])
    end
    return essays
end

essays = editEssays(essays)
array = editParagraphs(array)

# Question 9
authorSet = ["Hamilton", "Madison", "Jay", "Unknown", "All"]

function addEssaysByAuthor(essays, authors)
    essaysByAuthor = [String[], String[], String[], String[], String[]]
    for i in eachindex(authors)
        if(cmp(authors[i], "HAMILTON") == 0)
            push!(essaysByAuthor[1], essays[i])
        elseif(cmp(authors[i], "MADISON") == 0)
            push!(essaysByAuthor[2], essays[i])
        elseif(cmp(authors[i], "MADISON, WITH HAMILTON") == 0)
            push!(essaysByAuthor[1], essays[i])
            push!(essaysByAuthor[2], essays[i])
        elseif(cmp(authors[i], "JAY") == 0)
            push!(essaysByAuthor[3], essays[i])
        else
            push!(essaysByAuthor[4], essays[i])
        end
    end
    essaysByAuthor[5] = essays
    return essaysByAuthor
end

essaysByAuthor = addEssaysByAuthor(essays, authors)

# Question 9a
function avgWordsPerEssay(authorEssays)
    wordCount = 0
    for i in eachindex(authorEssays)
        wordCount += (count(" ", authorEssays[i]) + 1)
    end
    return wordCount / length(authorEssays)
end

for i in 1:5
    display("Number of words per essay by $(authorSet[i]): $(avgWordsPerEssay(essaysByAuthor[i]))")
end

function avgWordLength(authorEssays)

end

85-element Vector{SubString{String}}:
 "AFTER an unequivocal experienc" ⋯ 9392 bytes ⋯ "of my next address.\r\n\r\nPUBLIUS"
 "WHEN the people of America ref" ⋯ 10146 bytes ⋯ "ALL MY GREATNESS.\"\r\n\r\nPUBLIUS"
 "IT IS not a new observation th" ⋯ 8781 bytes ⋯ "er POWERFUL nation?\r\n\r\nPUBLIUS"
 "MY LAST paper assigned several" ⋯ 9762 bytes ⋯ "against themselves.\r\n\r\nPUBLIUS"
 "QUEEN ANNE, in her letter of t" ⋯ 8283 bytes ⋯ "of foreign nations.\r\n\r\nPUBLIUS"
 "THE three last numbers of this" ⋯ 13164 bytes ⋯ "ggests the REMEDY.\r\n\r\nPUBLIUS"
 "IT IS sometimes asked, with an" ⋯ 13912 bytes ⋯ "es or fears us.(2)\r\n\r\nPUBLIUS"
 "ASSUMING it therefore as an es" ⋯ 12232 bytes ⋯ "n, and formidable.\r\n\r\nPUBLIUS"
 "A FIRM Union will be of the ut" ⋯ 12103 bytes ⋯ " erroneous theory.\r\n\r\nPUBLIUS"
 "AMONG the numerous advantages " ⋯ 18118 bytes ⋯ "er of Federalists.\r\n\r\nPUBLIUS"
 ⋮
 "IT HAS been mentioned as one o" ⋯ 11772 bytes ⋯ "reasonable people?\r\n\r\nPUBLIUS"
 "WE PROCEED